# Code and Problem Diary

### Date: 17/04/2018, Author: MM

**Description of problem:**
Main issue trying to sort out at a high level is the training algorithm not converging... Assuming the algorithm is implemented correctly then there are two levers by which to alter the convergence of the algorith - the $\tau$ parameter controlling level of sparsity induced by the soft thresholding algorithm and the learning rate of the SGD controlling the step take at each iteration to update the dictionary. Currently we observe the following:
- If we set both $\tau$ and the learning rate to 1 then in the first iteration the sparsity level almost instantly falls to zero. Essentially the l1 error significance is so much larger than the l2 error that the minimum is found by selecting the minimiser of the l1 norm, the zero vector. If this happens in the first iteration then nothing else really matters, our algorithm has reached an equilibrium point from which it cannot escape.
- With a $\tau$ of order $10^{-5}$ and the learning rate still at 1 we finally get a setup where we do not almost immediatly plunge towards the 0 vector. Here we can observe that the l1 error even at 50% sparsity is around 390k while the l2 error term is only around 89k. Hence it is clear why the algorithm is prioritising minimising the l1 error. Interestingly after just two iterations of the whole algorith, e.g. FISTA then dictionary update we reach another equilibrium point, the sparsity level stays at around 63%, while the l2 error increases from around 87k untill it hits around 92k. This implies the learning rate is too high. The stepsize also becomes increasingly small which implies that since $\tau$ is fixed, 1/L is getting smaller and that hence L is getting large. This means that the largest singular value of the weight matrix is getting increasingly large, perhaps implying that the coefficient values are becoming increasingly large in the weight matrix. This is further evidence that the learning rate is to high and suggests and interestingly that the learning rate needs to be low enough to ensure that L is not to large. Furthermore it might suggest we should do something in terms of normalising the kernel filters in some way. This though one would think interrupts with the parameter learning
- Following on from the following bullet point if we use $\tau$ of order $10^{-5}$ and the learning rate at 0.0001 then the average loss per data point and the loss at each iteration at FISTA sticks. Although one would expect the l2 error to stay the same or maybe even increase within each FISTA step one would expect the error between each round of FISTA to decrease, since between each round the dictionary update should be working exclusively to reduce this error. However clearly the dictionary is failing to adapt to reduce the l2 error and converge to a lower value even with a very small step size. If we boost the learning rate to something silly like 10 then we instantly get back to the state we were in before where we sit at around 63% sparsity but with an l2 error of around 121k.

Big problem then seems to be in the dictionary update rather than the FISTA step. Observing the weight update it is clear that the weights are updating overtime, and also as might have been feared, that both $D$ and $D^T$ are consistent. Of note however is that the magnitudes of the weight elements is very small, order $10^{-5}$. Interestingly with tau set to 0, i.e. with no sparisty induced, neither the FISTA or the dictionary update reduce the l2 error term. This is quite curious - intuitively perhaps if the values of the weight matrix are very off then it will make it hard to learn a better X and likewise the converse. However this is far from a rigorously thought through statement! It seems unlikely that Pytorch would incorrectly implement e.g. SGD and code logic seems reasonable, so most likely case is that the FISTA implementation is not correct or we need to use some form of backtracking to update tau to deal with the change in the size of the norm. Next steps:
- Implement IHT with k = 20 and see if this works any better
- On FISTA try normalising the norms so that they are of equal importance/ controlled fully by $\tau$



### Date: 19/04/2018, Author: MM

**Description of problem:**
Have now implemented an IHT version of the forward pass single layer network. This seems to work a better but still is not perfect:
- With IHT as implemented after a few batches the average l2 error per image is around 0.025-0.04
- The FISTA model jumps consistently between 0.075 and 0.1

In both case decreasing the learning rate and increasing the number of steps at each dictionary update seems to provide significant benefits over the training over a given batch. However between each batch the l2 reconstruction error jumps back up, in. the case of FISTA to around 0.1 and in the case of IHT to around 0.06.

The big problem iwth FISTA seems to be the $\tau$ parameter; at the beginning if the tau parameter is too large then it simply sets the representation to 0. This is because the singular values of the weight matrix must be very small which makes sense for a random initialisation. However as the training progresses the L values grow and hence the stepsize $\tau/ L$ becomes entirely innapropriate. At this point sparsity fails to be enforced / encouraged so the sparsity level stays at around 64%. Oddly enough though the l2 error remains pretty high, around the 80-90k mark total which is curious, it should be able to get a lower l2 error than IHT since it is essentially only optimising over the l2. This happens because the update for for the ST argument is multiplied by the step size as well remember! This is very very small, so essentially we are just thresholding the same tensor, with the same threshold value. As a result because the stepsize essentially goes to zero then FISTA gets stuck in a cycle. ** *To fix the FISTA implementation need to update or manage the step size better with something like e.g. backtracking* **.

With regards to what is stopping IHT progressing So what happens is the following:
- a batch comes in and the 'forward pass' solves a sparse coding problem to generate a sparse representation. The algorithm for doing this is iterative; with FISTA we iteratively converge towards the minimum of the convex non-smooth objective function, with the IHT as implemented we fix the number of nonzeros and iterate to improve the l2 error on the reconstruction (I assume?!). Weirdly however at the moment although for the first few batches the l2 error decreases, after a few batches the l2 error increases over the iterations of the IHT! Something is amiss there
- For this given batch then we fix the representation that we found through the sparse coding and try to optimise the weights to improve the l2 error on the reconstruction. We do this by running a number of gradient descent steps, for both IHT and FISTA this step seems to work well! The error decreases a lot. At the end the weights for the forward and backward pass are updated to be consistent
- then a new batch comes in, the dictionary naturally hasn't been optimised for this batch, so the initial error is higher than it was at the end of the last batch. Run the sparse coding again etc. and continue.

The idea is that overtime the distance between the error at the end of the last dictionary update and the initial error at the beginning of the next batch converges towards 0. ** *To fix the IHT need to solve* **:
- the initial error for each new batch keeps jumping up again. A number of solutions for this are the following,  increase batch size, just train for longer, make sure not overfitting the dictionary as it where to a particular batch. ** *The most promising lines then is regard seem be just increase batch size and train for longer then, once we have a really low l2 error can worry about generalisation!* **
- ** *IHT error seems to be increasing with each iteration! This needs to be fixed* **


Problem identifief with the hardthresholding function, have not searched over the largest absolute values.


**Post chat with JT**
After chatting with Jared next steps:
- Normalise the filters so that they have l2 norm of 1. This should not affect things since when the sparse code is next computed it should be taken care of.
- FISTA: add backtracking on the lambda variable, ask Jere again what he did here, trying to recreate Jere's results
- IHT: still need to ensure that error is not increasing with each iteration, this may be due to the stepsize.
- Implement NIHT: need to calculate gamma term each time, may best to have as one combined class

Problem: now that I have normalised the atoms of the dictionary to be one, the IHT algorithm seems to explode... Observing the l2 norm of each weight it is 1 as intended. What is essentially happening is that the l2 error of the reconstruction is exploding with each iteration of either FISTA or NIHT. Printing out the largest coefficient per image across all filters the size of the filter grows exponentially per iteration. i.e. every iteration the magnitude of X is growing. Printing out the l2 norm of the sparse rep of a given image it grows exponentially. starting at 0 after a few iterations its around 700. It is also not just IHT, FISTA estimate also seems to explode.

### Date: 20/04/2018, Author: MM

Main effort this morning is focused around cracking the explosion in the l2 error of the signals X. Not entirely sure why the normalisation is causing such problems, believe the issue lies with FISTA and IHT implementations. The actual implementations seem to be ok: hence I think it is concerned with the values of the hyperparameters, i.e. the step sizes.

Note that for IHT the algorithm is only guaranteed to converge if the operator norm is less than one, which is not the case for normalised columns. A better more general algorithm to leverage is NIHT, where we apply a step size calculated in a specific way. Therefore plan is to implement NIHT and FISTA with backtracking to see if this solves the l2 error explosion. 

After implementing backtracking on FISTA seems to instantly set everything to zero. The cause for this is simple, randomly initialising X means that both the reconstruction error and the l1 norm component are high. Sadly with x set to zero the l2 error and the l1 error are lower, so it instantly jumps to this since, if we use Armijio method, we start at a high stepsize and stop when the error is lower than currently. As a result for an X that is initialised very small, any large value of alpha will zero all entries of X, and this new X will be selected. Once this X is zero it is hard to escape since our alpha gets progressively smaller and smaller and hence the updated attempt of try of X gets closer and closer to 0.

### Date 23/04/2018, Author: MM

Today aiming to sort out FISTA and NIHT. Observing and running FISTA it seems that the sparse coding step is sort of working... with each iteration of FISTA the l1 and l2 error seem to be decreasing. Across the six iterations of FISTA it seems big improvements are made, e.g. 1.3M at beginning for l2 error and 59K, sparsity level has also dropped to around 14%. Unfortunatly the dictionary update step does not seem to work. Increasing the learning rate helps significantly, but there seems to be this risk of FISTA still finding the zero solution. If the learning rate is increased the FISTA seems to find the zero vector on the second batch... Why might this be true? Fund a bug, had not updated the argument sequence so was recieving a tau of around

What seems tricky about these approaches is that it is hard to tell weather you have just coded somethign poorly or weather the hyperparameters are just a way off. 

### Date 27/04/2018, Author: MM

Have tried to implement OMP, with the X update solved via SGD. Two problems that I have observed are:
- Firstly one cannot select to apply a filter at a specific location, the conv operator applies any filter in its weights across all locations. This is not an issue currently as we are using fully connected layers but we might want to think about it in future
- Secondly the residue as each filter is added is barely reducing. This is probably because all the filters at this stage are pretty much random noise, and therefore none correlate very highly with it. The atom's contribution has not been entirely removed though unfortunatly. It seems to therefoe re-select this atom when choosing the next atom. If wee solved the OMP properly this shouldn't happen since the residue is orthogonal to the support dictionary. In other words the residue is in the Null space of the support dictionary and is hence orthogonal to the column space of the support dictionary.

Second issue is the one I am trying to resolve. Interestingly the SGD seems to have converged to a loss of around 0.065; it does not seem to be the case that it is just that we are not properly solving for x and hence some of the residual is in part still present next iteration to correlate highly with the same atom again. Indeed, if this were the case then SGD would still to be seen to be decreasing. As a result it must be that there is another issue occurring... On analysis the restructured version Y formed from the support x and support dictionary is different from the full dictionary and the support x embedded in a tensor that is mostly zeros. 